# Memory MCP Lab - Testing mcp-memory-libsql API

本 Notebook 用於驗證 `memory_tools.py` 中的 `memory_mcp.session.call_tool()` 使用方式，
以及理解 mcp-memory-libsql 提供的 API 工具，包括：
- `create_entities`: 建立記憶體實體
- `search_nodes`: 搜尋節點
- `read_graph`: 讀取完整圖表
- `delete_entity`: 刪除實體

目標：透過實際測試來驗證 memory_tools.py 中的函數邏輯是否正確。

## Section 1: 環境設置與依賴安裝

In [5]:
import os
import sys
from pathlib import Path
from dotenv import dotenv_values
import asyncio
import json
from contextlib import AsyncExitStack
from datetime import datetime

# 設定 Python 路徑，以便導入專案模組
project_root = Path.cwd().parent.parent.parent
sys.path.insert(0, str(project_root / "backend" / "src"))

# 找到 .env 檔案
env_file = project_root / "backend" / ".env"
if not env_file.exists():
    env_file = Path.home() / "Documents/workspace/CasualTrader/backend/.env"

# 讀取並設定環境變數
if env_file.exists():
    env_vars = dotenv_values(env_file)
    for key, value in env_vars.items():
        os.environ[key] = value.strip('"').strip("'") if value else ""
    print(f"✓ .env 已載入: {env_file}")
else:
    print(f"✗ .env 檔案未找到: {env_file}")

# 驗證必要的環境變數
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"✓ OPENAI_API_KEY: {'已設定' if OPENAI_API_KEY else '未設定'}")


✓ .env 已載入: /Users/sacahan/Documents/workspace/CasualTrader/backend/.env
✓ OPENAI_API_KEY: 已設定


## Section 2: MCP Memory LibSQL 基本概念

MCP (Model Context Protocol) Memory LibSQL 是一個為 AI 模型提供持久化記憶體的 MCP 伺服器。
它使用 LibSQL (SQLite 的輕量級衍生版本) 作為後端存儲。

### 核心特性：
1. **Entity 實體管理**：儲存帶有觀察值的命名實體
2. **關係管理**：定義實體之間的連接
3. **向量搜尋**：支援語義搜尋功能
4. **圖表操作**：完整的知識圖表管理

### 主要工具：
- `create_entities`: 建立或更新實體及其觀察
- `search_nodes`: 基於查詢搜尋相似節點
- `read_graph`: 讀取完整或部分圖表
- `delete_entity`: 刪除實體及其關聯

## Section 3: 初始化 Memory MCP 連接

In [6]:
from agents.mcp import MCPServerStdio
import logging

# 設置日誌
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# 全域變數用於存儲 MCP 伺服器實例和 exit stack
memory_mcp = None
_exit_stack = None

async def initialize_memory_mcp():
    """
    初始化 Memory MCP 伺服器
    
    Returns:
        MCPServerStdio: Memory MCP 伺服器實例
    """
    global memory_mcp, _exit_stack
    
    try:
        if not _exit_stack:
            _exit_stack = AsyncExitStack()
        
        # 配置 LibSQL 資料庫路徑
        memory_db_path = project_root / "backend" / "memory" / "test_memory.db"
        memory_db_path.parent.mkdir(parents=True, exist_ok=True)
        
        print("📦 初始化 Memory MCP...")
        print(f"   Database: {memory_db_path}")
        
        memory_mcp = await _exit_stack.enter_async_context(
            MCPServerStdio(
                name="memory_mcp",
                params={
                    "command": "npx",
                    "args": ["-y", "mcp-memory-libsql"],
                    "env": {"LIBSQL_URL": f"file:{memory_db_path}"},
                },
                client_session_timeout_seconds=30.0,
            )
        )
        
        print("✓ Memory MCP 初始化完成")
        return memory_mcp
    
    except Exception as e:
        print(f"✗ 初始化失敗: {type(e).__name__}: {str(e)}")
        if _exit_stack:
            await _exit_stack.aclose()
            _exit_stack = None
        raise

async def list_available_tools():
    """
    列出 Memory MCP 提供的所有工具
    """
    global memory_mcp
    
    try:
        if not memory_mcp:
            print("✗ Memory MCP 未初始化")
            return []
        
        # 列出可用工具
        tools_result = await memory_mcp.session.list_tools()
        tools = tools_result.tools if hasattr(tools_result, 'tools') else []
        
        print("\n📋 Memory MCP 可用工具:")
        print("-" * 60)
        for tool in tools:
            print(f"  • {tool.name}")
            if hasattr(tool, 'description'):
                print(f"    描述: {tool.description}")
        
        return tools
    
    except Exception as e:
        print(f"✗ 列出工具失敗: {type(e).__name__}: {str(e)}")
        return []

# 執行初始化
print("=" * 60)
print("正在初始化 Memory MCP...")
print("=" * 60)

memory_mcp = await initialize_memory_mcp()
tools = await list_available_tools()


正在初始化 Memory MCP...
📦 初始化 Memory MCP...
   Database: /Users/sacahan/Documents/workspace/CasualTrader/backend/memory/test_memory.db
✓ Memory MCP 初始化完成

📋 Memory MCP 可用工具:
------------------------------------------------------------
  • create_entities
    描述: Create new entities with observations
  • search_nodes
    描述: Search for entities and their relations using text search with relevance ranking
  • read_graph
    描述: Get recent entities and their relations
  • create_relations
    描述: Create relations between entities
  • delete_entity
    描述: Delete an entity and all its associated data (observations and relations)
  • delete_relation
    描述: Delete a specific relation between entities
✓ Memory MCP 初始化完成

📋 Memory MCP 可用工具:
------------------------------------------------------------
  • create_entities
    描述: Create new entities with observations
  • search_nodes
    描述: Search for entities and their relations using text search with relevance ranking
  • read_graph
    描述: Get 

## Section 4: 測試 create_entities 工具

In [ ]:
async def test_create_entities():
    """
    測試 create_entities 工具
    
    此工具用於建立或更新記憶體中的實體。
    響應格式：包含 content 列表，其中第一項是 TextContent
    """
    global memory_mcp
    
    if not memory_mcp:
        print("✗ Memory MCP 未初始化")
        return
    
    print("\n" + "=" * 60)
    print("測試: create_entities")
    print("=" * 60)
    
    try:
        # 建立測試實體
        test_entity = {
            "name": "agent_test_123_execution_2025",
            "entityType": "trading_execution",
            "observations": [
                "Mode: AUTO_TRADING",
                "Buy TSMC 2330 100 shares",
            ]
        }
        
        print("\n📤 發送請求:")
        print("   工具: create_entities")
        print(f"   實體: {test_entity['name']}")
        print(f"   類型: {test_entity['entityType']}")
        
        result = await memory_mcp.session.call_tool(
            "create_entities",
            {
                "entities": [test_entity]
            }
        )
        
        # 解析回應
        print(f"\n📥 收到回應:")
        print(f"   類型: {type(result)}")
        print(f"   屬性: {dir(result)}")
        
        if hasattr(result, "content") and result.content:
            print(f"   Content 類型: {type(result.content)}")
            print(f"   Content 長度: {len(result.content)}")
            
            for i, content_item in enumerate(result.content):
                print(f"\n   [Content {i}]")
                print(f"     類型: {type(content_item)}")
                print(f"     屬性: {dir(content_item)}")
                
                if hasattr(content_item, "text"):
                    print(f"     文本: {content_item.text[:200]}...")
                elif hasattr(content_item, "value"):
                    print(f"     值: {content_item.value}")
        else:
            print(f"   無 content 屬性或為空")
        
        print(f"\n✓ create_entities 測試完成")
        return result
    
    except Exception as e:
        print(f"\n✗ 錯誤: {type(e).__name__}: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 執行測試
result_create = await test_create_entities()


## Section 5: 測試 search_nodes 工具

## Entity, Observation, Relation 之間的關係

根據官方源碼分析，mcp-memory-libsql 的數據模型採用**知識圖譜（Knowledge Graph）**結構。

### 🔗 完整關係示意

```
┌─────────────────────────────────────────────────────────────────┐
│                    ENTITY (實體/節點)                            │
│  name: "trading_decision_2025_01_15"                            │
│  type: "trading_decision"                                       │
│  embedding: [0.1, 0.2, ...]                                     │
└────────┬─────────────────────────────┬─────────────────────────┘
         │                             │
         │ (1:N 關係)                  │ (N:N 關係)
         │                             │
    ┌────v──────────────┐         ┌────v────────────────┐
    │  OBSERVATIONS     │         │   RELATIONS         │
    ├───────────────────┤         ├─────────────────────┤
    │ id: 1             │         │ id: 1               │
    │ content: "買入..."│         │ source: this entity │
    ├───────────────────┤         │ target: other entity│
    │ id: 2             │         │ type: "based_on"    │
    │ content: "目標..."│         └─────────────────────┘
    ├───────────────────┤
    │ id: 3             │
    │ content: "技術..."│
    └───────────────────┘
```

### 💡 三者的角色定義

| 組件 | 用途 | 例子 |
|-----|------|------|
| **Entity** | 知識圖的**節點** | "TSMC股票", "買入決策", "市場分析" |
| **Observation** | Entity 的**屬性或事實** | "TSMC股價 615元", "技術指標看好" |
| **Relation** | Entity 間的**連接** | "決策基於分析", "結果來自決策" |

### 🔍 實際工作流程示例

```python
# 1. 創建第一個實體 + 觀測
await memory_mcp.session.call_tool(
  "create_entities",
  {
    "entities": [{
      "name": "market_analysis_2025_01_15",      # Entity
      "entityType": "market_analysis",
      "observations": [                          # Observations
        "台灣股市開盤 17,500",
        "半導體板塊漲幅 2.3%",
        "TSMC 出現買入機會"
      ]
    }]
  }
)

# 2. 創建第二個實體 + 觀測 + 關係
await memory_mcp.session.call_tool(
  "create_entities",
  {
    "entities": [{
      "name": "trading_decision_2025_01_15",     # Entity
      "entityType": "trading_decision",
      "observations": [                          # Observations
        "基於市場分析決定買入",
        "TSMC 2330 目標價 620 元",
        "買入 1000 股"
      ],
      "relations": [{                            # Relations
        "target": "market_analysis_2025_01_15",
        "relationType": "based_on"
      }]
    }]
  }
)

# 3. 查詢時，會返回完整的關係圖
result = await memory_mcp.session.call_tool("read_graph", {})
# 返回:
# {
#   "nodes": [
#     {entity data},
#     {entity data}
#   ],
#   "relations": [
#     {relation data}
#   ]
# }
```


## 按 entity_type 進行查詢

### 問題：能否直接指定 entity_type 過濾？

根據官方源碼分析，`search_nodes` **沒有直接的 `entity_type` 參數**，但可以通過查詢 entity_type 值間接實現：

### 官方源碼中的搜尋邏輯

```sql
-- 文本搜尋時，會在以下三個字段中搜尋：
WHERE e.name LIKE ? 
  OR e.entity_type LIKE ? 
  OR o.content LIKE ?
```

也就是說，查詢語句會搜索：
- `Entity.name`：實體的唯一名稱
- `Entity.entity_type`：實體的類型字段 ✅
- `Observation.content`：觀測內容

### 按 entity_type 查詢的方法

#### 方式 1：直接查詢 entity_type 名稱（推薦）

```python
# 查詢所有 "trading_decision" 類型的實體
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {
        "query": "trading_decision",  # 這會匹配 entity_type="trading_decision"
        "limit": 10
    }
)

data = json.loads(result.content[0].text)
entities = data["nodes"]
# 返回的實體會包括所有 entity_type 為 "trading_decision" 的實體
```

#### 方式 2：結合 entity_type 和其他信息

```python
# 查詢 "trading" 類型中包含 "TSMC" 的實體
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {
        "query": "trading_decision TSMC",  # 會匹配包含這些詞的實體
        "limit": 10
    }
)
```

#### 方式 3：在應用層進行過濾（精確匹配）

如果需要精確匹配 `entity_type`，可以在應用層過濾結果：

```python
async def search_by_entity_type(memory_mcp, entity_type: str, limit: int = 10):
    """
    按 entity_type 精確查詢
    """
    # 查詢 entity_type
    result = await memory_mcp.session.call_tool(
        "search_nodes",
        {
            "query": entity_type,
            "limit": limit
        }
    )
    
    data = json.loads(result.content[0].text)
    entities = data["nodes"]
    
    # 應用層過濾：只返回精確匹配的實體
    filtered = [e for e in entities if e.get("entityType") == entity_type]
    
    return {
        "nodes": filtered,
        "relations": data["relations"]
    }

# 使用
result = await search_by_entity_type(memory_mcp, "trading_decision")
```

### 實際例子

```python
# 假設已創建以下實體：
# 1. market_analysis_2025_01_15 (entityType: "market_analysis")
# 2. trading_decision_2025_01_15 (entityType: "trading_decision")
# 3. trading_result_2025_01_15 (entityType: "trading_result")

# 查詢所有交易決策
result = await memory_mcp.session.call_tool(
    "search_nodes",
    {"query": "trading_decision"}
)

data = json.loads(result.content[0].text)
print(f"找到 {len(data['nodes'])} 個交易決策實體")

# 輸出示例：
# 找到 1 個交易決策實體
# {
#   "nodes": [
#     {
#       "name": "trading_decision_2025_01_15",
#       "entityType": "trading_decision",
#       "observations": [...],
#       ...
#     }
#   ]
# }
```

### 重要限制

| 限制項 | 說明 |
|-------|------|
| **最多 5 個結果** | 無論 `limit` 參數是多少，最多返回 5 個結果（硬限制） |
| **模糊匹配** | 使用 LIKE %query%，即子字符串匹配，不是精確匹配 |
| **搜索範圍** | 同時搜索 name、entity_type 和 observations |
| **無過濾器參數** | search_nodes 本身沒有 `filter` 或 `entityType` 參數 |

### 最佳實踐

1. **為 entity_type 使用有意義的名稱**
   ```python
   # ✅ 好的命名
   "entityType": "trading_decision"
   "entityType": "market_analysis"
   
   # ❌ 容易混淆的命名
   "entityType": "t1"
   "entityType": "m1"
   ```

2. **使用詳細的搜尋詞**
   ```python
   # ✅ 更容易找到
   await memory_mcp.session.call_tool(
       "search_nodes",
       {"query": "trading_decision TSMC 2330"}
   )
   
   # ⚠️ 可能與其他實體混淆
   await memory_mcp.session.call_tool(
       "search_nodes",
       {"query": "trading"}
   )
   ```

3. **在應用層維護索引**
   如果需要頻繁按 entity_type 查詢，建議在應用層維護一個索引：
   ```python
   # 初始化時構建索引
   async def build_entity_type_index(memory_mcp):
       result = await memory_mcp.session.call_tool("read_graph", {})
       data = json.loads(result.content[0].text)
       
       entity_type_index = {}
       for entity in data["nodes"]:
           entity_type = entity.get("entityType")
           if entity_type not in entity_type_index:
               entity_type_index[entity_type] = []
           entity_type_index[entity_type].append(entity.get("name"))
       
       return entity_type_index
   ```

In [12]:
global memory_mcp

print("\n" + "=" * 80)
print("測試: 按 entity_type 進行查詢")
print("=" * 80)

# 方式 1: 直接查詢 entity_type
print("\n\n1️⃣  方式 1: 直接查詢 entity_type 名稱")
print("-" * 80)

entity_type = "trading_decision"

print(f"\n查詢: '{entity_type}'")
try:
    result = await memory_mcp.session.call_tool(
        "search_nodes",
        {
            "query": entity_type, 
            "limit": 10
        }
    )

    print("  查詢完成 ✓")
    print("#" * 80)
    print(f"{result.content[0].text}")
    print("#" * 80)
    
    if hasattr(result, "content") and result.content:
        content_item = result.content[0]
        text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
        
        # 解析 JSON
        data = json.loads(text_content)
        print(f"  JSON 鍵: {list(data.keys())}")
        
        # 注意：search_nodes 返回的是 "entities"，不是 "nodes"
        entities = data.get("entities", [])
        relations = data.get("relations", [])
        
        print(f"\n  找到 {len(entities)} 個實體:")
        for entity in entities:
            print(f"    • {entity.get('name')} (type: {entity.get('entityType')})")
            observations = entity.get('observations', [])
            if observations:
                print(f"      觀測數: {len(observations)}")
                for obs in observations[:2]:  # 只顯示前2個觀測
                    print(f"        - {obs}")
            
except Exception as e:
    print(f"  ❌ 查詢失敗: {e}")
    import traceback
    traceback.print_exc()


測試: 按 entity_type 進行查詢


1️⃣  方式 1: 直接查詢 entity_type 名稱
--------------------------------------------------------------------------------

查詢: 'trading_decision'
  查詢完成 ✓
################################################################################
{
  "entities": [
    {
      "name": "trading_decision_buy_tsmc",
      "entityType": "trading_decision",
      "observations": [
        "買入 TSMC 2330",
        "數量 1000 股"
      ]
    },
    {
      "name": "trading_decision_sell_amd",
      "entityType": "trading_decision",
      "observations": [
        "賣出 AMD",
        "獲利了結"
      ]
    },
    {
      "name": "trading_decision_2025_01_15",
      "entityType": "trading_decision",
      "observations": [
        "時間: 2025-01-15 10:30",
        "決策: 買入 TSMC 2330",
        "數量: 1000 股",
        "目標價: 620 元",
        "技術指標: RSI 35(超賣), MACD 正向交叉",
        "理由: 基於市場分析和技術信號"
      ]
    }
  ],
  "relations": []
}
##########################################################################

In [ ]:
# 方式 2: 結合 entity_type 和其他條件
print("\n\n2️⃣  方式 2: 結合 entity_type 和其他條件")
print("-" * 80)

combined_queries = [
    "trading_decision TSMC",
    "trading_decision 賣出",
]

for query in combined_queries:
    print(f"\n查詢: '{query}'")
    try:
        result = await memory_mcp.session.call_tool(
            "search_nodes",
            {"query": query, "limit": 10}
        )
        
        if hasattr(result, "content") and result.content:
            content_item = result.content[0]
            text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
            data = json.loads(text_content)
            
            # 正確的鍵是 "entities"
            entities = data.get("entities", [])
            
            print(f"  找到 {len(entities)} 個結果:")
            for entity in entities:
                print(f"    • {entity.get('name')}")
                observations = entity.get('observations', [])
                if observations:
                    print(f"      > {observations[0]}")
                    
    except Exception as e:
        print(f"  ❌ 查詢失敗: {e}")

In [ ]:
# 方式 3: 應用層精確過濾
print("\n\n3️⃣  方式 3: 應用層精確過濾 (推薦用於精確匹配)")
print("-" * 80)

async def search_by_entity_type_exact(entity_type: str):
    """精確查詢指定 entity_type 的所有實體"""
    try:
        result = await memory_mcp.session.call_tool(
            "search_nodes",
            {"query": entity_type, "limit": 10}
        )
        
        if hasattr(result, "content") and result.content:
            content_item = result.content[0]
            text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
            data = json.loads(text_content)
            
            # 使用 "entities" 鍵，不是 "nodes"
            entities = data.get("entities", [])
            
            # 應用層過濾：只返回精確匹配的實體
            filtered = [e for e in entities if e.get("entityType") == entity_type]
            
            return {
                "query_entity_type": entity_type,
                "total_found": len(entities),
                "exact_matched": len(filtered),
                "entities": filtered
            }
    except Exception as e:
        print(f"  ❌ 查詢失敗: {e}")
        return None

print("\n精確查詢 'trading_decision' 類型的實體:")
result = await search_by_entity_type_exact("trading_decision")
if result:
    print(f"  查詢 entity_type: {result['query_entity_type']}")
    print(f"  搜尋結果總數: {result['total_found']}")
    print(f"  精確匹配數: {result['exact_matched']}")
    print(f"\n  結果詳細信息:")
    for entity in result['entities']:
        print(f"    • {entity.get('name')} (type: {entity.get('entityType')})")
        observations = entity.get('observations', [])
        print(f"      觀測數: {len(observations)}")
        for i, obs in enumerate(observations[:3], 1):  # 顯示前3個觀測
            print(f"      {i}. {obs}")

In [ ]:
# 驗證
print("\n\n4️⃣  驗證: 讀取完整圖譜確認")
print("-" * 80)

try:
    result = await memory_mcp.session.call_tool(
        "read_graph",
        {"limit": 100}
    )
    
    if hasattr(result, "content") and result.content:
        content_item = result.content[0]
        text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
        data = json.loads(text_content)
        
        # read_graph 返回的是 "nodes"
        nodes = data.get("nodes", [])
        relations = data.get("relations", [])
        
        # 統計每個 entity_type 的數量
        entity_type_count = {}
        for node in nodes:
            entity_type = node.get("entityType")
            entity_type_count[entity_type] = entity_type_count.get(entity_type, 0) + 1
        
        print(f"\n✅ 圖譜統計:")
        print(f"  總實體數: {len(nodes)}")
        print(f"  總關係數: {len(relations)}")
        print(f"\n  Entity Type 分佈:")
        for entity_type, count in sorted(entity_type_count.items()):
            print(f"    • {entity_type}: {count} 個實體")
            
except Exception as e:
    print(f"❌ 讀取圖譜失敗: {e}")

print(f"\n✅ 測試完成")

In [ ]:

# 診斷：檢查 DB 中的實際數據
import sqlite3

db_path = "/Users/sacahan/Documents/workspace/CasualTrader/backend/memory/test_memory.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print("\n📊 數據庫中的實體:")
cursor.execute("SELECT name, entity_type FROM entities LIMIT 15")
for row in cursor.fetchall():
    print(f"  • {row[0]} (type: {row[1]})")

conn.close()

# 嘗試 read_graph
print("\n🔍 嘗試 read_graph:")
try:
    result = await memory_mcp.session.call_tool("read_graph", {"limit": 100})
    if hasattr(result, "content") and result.content:
        content_item = result.content[0]
        text_content = content_item.text if hasattr(content_item, "text") else str(content_item)
        data = json.loads(text_content)
        print(f"  找到 {len(data.get('nodes', []))} 個實體")
except Exception as e:
    print(f"  ❌ 錯誤: {e}")
